## 3.3.4: Challenge (Logistic, Ridge, LASSO Regression)
## Kevin Hahn  

<b>In your report, evaluate all three models and decide on your best. Be clear about the decisions you made that led to these models (feature selection, regularization parameter selection, model evaluation criteria) and why you think that particular model is the best of the three. Also reflect on the strengths and limitations of regression as a modeling approach. Were there things you couldn't do but you wish you could have done?

Record your work and reflections in a notebook to discuss with your mentor.</b>

I looked at the Wisconsin breast cancer dataset (http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Original%29) and ran a standard logistic regression, ridge regression, and LASSO regression on the original 9 predictor variables ("few features") and also those original with 6 added features ("many features"). Features were formed by finding products of predictors strongly correlated with one another or with the target variable of stage 2/stage 4 cancer. L2 regularization was used for the "vanilla" logistic regression as least squares regression appeared to account for most variance in the target variable. 

Here were the R-squared values for the few and many feature set versions of the three regressions:

"Vanilla" logistic regression with few features, with many features:
0.985380116959, 0.753461134895

Ridge logistic regression with few features, with many features:
0.730833757103, 0.797642266828

LASSO logistic regression with few features, with many features:
0.722511709555, 0.753461134895

Standard logistic regression with the naturally-occurring features already present in the dataset seemed to account for the most variance. The high R-squared value found in the test set seems to parallel the accuracy of the original Logit regression model. 

I do wish I knew more about the dataset itself because it seems like some of the values assigned are scaled linearly or ordinally but it's hard to say if some of the values assigned are subjective or which features/predictor variables could further be removed.

In [407]:
## Final output
## Run me last 

print('Logistic Regression R² for the test data with few features:')
print(logisticRegFit.score(X_test, Y_test))
print('\nLogistic Regression R² for the test data with many features:')
print(logisticRegBigFit.score(X_test2, Y_test))

print("")
print("Ridge Regression R-squared for small model:")
print(ridgeregr.score(X_test, Y_test))
print("")
print("Ridge Regression R-squared for large model:")
print(ridgeregrBig.score(X_test2, Y_test))

print("")
print('LASSO Regression R² for the test data with few features:')
print(lass.score(X_test, Y_test))
print('\nLASSO Regression R² for the test data with many features:')
print(lassBig.score(X_test2, Y_test))

Logistic Regression R² for the test data with few features:
0.985380116959

Logistic Regression R² for the test data with many features:
0.753461134895

Ridge Regression R-squared for small model:
0.730833757103

Ridge Regression R-squared for large model:
0.797642266828

LASSO Regression R² for the test data with few features:
0.722511709555

LASSO Regression R² for the test data with many features:
0.753461134895


In [396]:
import math
from sklearn import cross_validation
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std

import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn import linear_model
%matplotlib inline
import scipy
import sklearn
import seaborn as sns
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")
%matplotlib inline
sns.set_style('white')

In [397]:
dataframe = pd.read_csv("~/Documents/cancer-wisc2.csv", header=None, dtype="int")

dataframe.columns = ['id', 'clump_thickness', 'cell_size', 'cell_shape', 'marginal_adhesion', 'epithelial_size', 'bare_nuclei', 'bland_chromatin', 'normal_nucleoli','mitoses', 'class']

dataframe = dataframe.replace(to_replace=-1, value=np.nan)

dataframe = dataframe.drop('id', 1)

dataframe.dropna()

dataframe.head()

names = dataframe.columns

In [398]:
dataframe.clump_thickness = dataframe.clump_thickness.apply(lambda x: float(x))
dataframe.cell_size = dataframe.cell_size.apply(lambda x: float(x))
dataframe.cell_shape = dataframe.cell_shape.apply(lambda x: float(x))
dataframe.marginal_adhesion = dataframe.marginal_adhesion.apply(lambda x: float(x))
dataframe.epithelial_size = dataframe.epithelial_size.apply(lambda x: float (x))
dataframe.bare_nuclei = dataframe.bare_nuclei.apply(lambda x: float(x))
dataframe.bland_chromatin = dataframe.bland_chromatin.apply(lambda x: float(x))
dataframe.normal_nucleoli = dataframe.normal_nucleoli.apply(lambda x: float(x))
dataframe.mitoses = dataframe.mitoses.apply(lambda x: float(x))
dataframe['class'] = dataframe['class'].apply(lambda x: float(x))
dataframe = dataframe.dropna()

In [399]:
X = dataframe[['clump_thickness', 'cell_size', 'cell_shape', 
               'marginal_adhesion', 'epithelial_size', 'bare_nuclei', 
               'bland_chromatin', 'normal_nucleoli','mitoses']]
Y = dataframe['class']

Y = Y.apply(lambda x: 1 if x > 3 else 0)
dataframe['class'] = dataframe['class'].apply(lambda x: 1 if x > 3 else 0)

In [400]:
dataframe.head(10)

,clump_thickness,cell_size,cell_shape,marginal_adhesion,epithelial_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,5.000,1.000,1.000,1.000,2.000,1.000,3.000,1.000,1.000,0
1,5.000,4.000,4.000,5.000,7.000,10.000,3.000,2.000,1.000,0
2,3.000,1.000,1.000,1.000,2.000,2.000,3.000,1.000,1.000,0
3,6.000,8.000,8.000,1.000,3.000,4.000,3.000,7.000,1.000,0
4,4.000,1.000,1.000,3.000,2.000,1.000,3.000,1.000,1.000,0
5,8.000,10.000,10.000,8.000,7.000,10.000,9.000,7.000,1.000,1
6,1.000,1.000,1.000,1.000,2.000,10.000,3.000,1.000,1.000,0
7,2.000,1.000,2.000,1.000,2.000,1.000,3.000,1.000,1.000,0
8,2.000,1.000,1.000,1.000,2.000,1.000,1.000,1.000,5.000,0
9,4.000,2.000,1.000,1.000,2.000,1.000,2.000,1.000,1.000,0


In [401]:
# Declare predictors.
X_statsmod = X

# The Statsmodels formulation requires a column with constant value 1 that
# will act as the intercept.
X_statsmod['intercept'] = 1 

# Declare and fit the model.
logit = sm.Logit(Y, X_statsmod)
result = logit.fit()

# Lots of information about the model and its coefficients, but the
# accuracy rate for predictions is missing.
print(result.summary())

# Calculate accuracy. First, get probability that each row will be admitted.
pred_statsmod = result.predict(X_statsmod)

# Code admission as 1 if probability is greater than .5.
pred_y_statsmod = np.where(pred_statsmod < .5, 0, 1)

# Accuracy table.
table = pd.crosstab(Y, pred_y_statsmod)

print('\n Accuracy by Stage 4 cancer status')
print(table)
print('\n Percentage accuracy')
print((table.iloc[0,0] + table.iloc[1,1]) / (table.sum().sum()))

Optimization terminated successfully.
         Current function value: 0.075321
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                  class   No. Observations:                  683
Model:                          Logit   Df Residuals:                      673
Method:                           MLE   Df Model:                            9
Date:                Tue, 09 May 2017   Pseudo R-squ.:                  0.8837
Time:                        18:24:09   Log-Likelihood:                -51.444
converged:                       True   LL-Null:                       -442.18
                                        LLR p-value:                2.077e-162
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
clump_thickness       0.5350      0.142      3.767      0.000       0.257       0.813
cell_si

In [402]:
# Define the training and test sizes.
trainsize = int(dataframe.shape[0] / 2)
df_test = dataframe.iloc[trainsize:, :].copy()
df_train = dataframe.iloc[:trainsize, :].copy()

# Set up the regression model to predict solar radiation using all other
# variables as features.
regr1 = linear_model.LinearRegression()
Y_train = df_train['class'].values.reshape(-1, 1)
X_train = df_train.loc[:, ~(df_train.columns).isin(['class'])]
regr1.fit(X_train, Y_train)
print('\nR-squared simple model (TRAIN data):')
print(regr1.score(X_train, Y_train))

#Store the parameter estimates.
origparams = np.append(regr1.coef_, regr1.intercept_)

# Make new features to capture potential quadratic and cubic relationships
df_train['cell_size_by_bland_chromatin'] = df_train['cell_size'] * df_train['bland_chromatin']
df_train['cell_shape_by_cell_size'] = df_train['cell_shape'] * df_train['cell_size']
df_train['cell_shape_by_bland_chromatin'] = df_train['cell_shape'] * df_train['bland_chromatin']
df_train['bare_nuclei^2'] = df_train['bare_nuclei'] ** 2
df_train['mitoses^2'] = df_train['mitoses'] ** 2
df_train['mitoses^3'] = df_train['mitoses'] ** 3

# Re-run the model with the new features.
regrBig = linear_model.LinearRegression()
X_train2 = df_train.loc[:, ~(df_train.columns).isin(['class'])]
regrBig.fit(X_train2, Y_train)
print('\nR-squared complex model (TRAIN data):')
print(regrBig.score(X_train2, Y_train))

# Store the new parameter estimates for the same features.
newparams = np.append(
    regrBig.coef_[0,0:(len(origparams)-1)],
    regrBig.intercept_)

print('\nParameter Estimates for the same predictors for the small model '
      'and large model (TRAIN data):')
compare = np.column_stack((origparams, newparams))
prettycompare = np.array2string(
    compare,
    formatter={'float_kind':'{0:.3f}'.format})
print(prettycompare)
print("")

# Test the simpler model with smaller coefficients.
Y_test = df_test['class'].values.reshape(-1, 1)
X_test = df_test.loc[:, ~(df_test.columns).isin(['class'])]
print('\nR-squared simple model (test data):')
print(regr1.score(X_test, Y_test))

# Make new features to capture potential quadratic and cubic relationships
df_test['cell_size_by_bland_chromatin'] = df_test['cell_size'] * df_test['bland_chromatin']
df_test['cell_shape_by_cell_size'] = df_test['cell_shape'] * df_test['cell_size']
df_test['cell_shape_by_bland_chromatin'] = df_test['cell_shape'] * df_test['bland_chromatin']
df_test['bare_nuclei^2'] = df_test['bare_nuclei'] ** 2
df_test['mitoses^2'] = df_test['mitoses'] ** 2
df_test['mitoses^3'] = df_test['mitoses'] ** 3

# Re-run the model with the new features.
X_test2 = df_test.loc[:, ~(df_test.columns).isin(['class'])]
print('\nR-squared complex model (test data):')
print(regrBig.score(X_test2, Y_test))


R-squared simple model (TRAIN data):
0.808576784249

R-squared complex model (TRAIN data):
0.845728517278

Parameter Estimates for the same predictors for the small model and large model (TRAIN data):
[[0.042 0.029]
 [0.005 0.086]
 [0.023 0.087]
 [0.016 0.015]
 [0.015 0.012]
 [0.044 0.081]
 [0.007 0.020]
 [0.015 0.005]
 [0.003 -0.015]
 [-0.225 -0.400]]


R-squared simple model (test data):
0.845511518194

R-squared complex model (test data):
0.867105297604


In [403]:
# Fitting a ridge regression model. Alpha(lambda) is the regularization
# parameter (usually called lambda). As alpha gets larger, parameter
# shrinkage grows more pronounced. Note that by convention, the
# intercept is not regularized. Since we standardized the data
# earlier, the intercept should be equal to zero and can be dropped.

ridgeregr = linear_model.Ridge(alpha=12, fit_intercept=False, normalize=False) 
ridgeregr.fit(X_train, Y_train)
print('\nRidge Regression simple model (TRAIN data):')

print(ridgeregr.score(X_train, Y_train))
print("")
origparams = ridgeregr.coef_[0] #leaving out intercept
print(origparams)

ridgeregrBig = linear_model.Ridge(alpha=12, fit_intercept=False, normalize=True)
ridgeregrBig.fit(X_train2, Y_train)
print("")
print('\nRidge Regression complex model (TRAIN data):')
print(ridgeregrBig.score(X_train2, Y_train))
newparams = ridgeregrBig.coef_[0, 0:len(origparams)] #leaving out intercept

print('\nParameter Estimates for the same predictors for the small model'
      ' and large model:')
compare = np.column_stack((origparams, newparams))
prettycompare = np.array2string(
    compare,
    formatter={'float_kind':'{0:.3f}'.format})
print(prettycompare)
print("")
print("Ridge Regression R-squared for small model:")
print(ridgeregr.score(X_test, Y_test))
print("")
print("Ridge Regression R-squared for large model:")
print(ridgeregrBig.score(X_test2, Y_test))


Ridge Regression simple model (TRAIN data):
0.772291783965

[ 0.026574    0.0213824   0.02412437  0.01538304 -0.00288774  0.0466791
 -0.02274711  0.01858793 -0.00232403]


Ridge Regression complex model (TRAIN data):
0.823065069254

Parameter Estimates for the same predictors for the small model and large model:
[[0.027 0.030]
 [0.021 0.072]
 [0.024 0.073]
 [0.015 0.012]
 [-0.003 0.005]
 [0.047 0.037]
 [-0.023 -0.028]
 [0.019 0.013]
 [-0.002 -0.219]]

Ridge Regression R-squared for small model:
0.730833757103

Ridge Regression R-squared for large model:
0.797642266828


In [404]:
# LASSO Regression Model

# Small number of parameters.
lass = linear_model.Lasso(alpha=.35, normalize=False)
lassfit = lass.fit(X_train, Y_train)
print('R² for the model with few features:')
print(lass.score(X_train, Y_train))
origparams = np.append(lassfit.coef_, lassfit.intercept_)
print('\nParameter estimates for the model with few features:')
print(origparams)

# Large number of parameters.
lassBig = linear_model.Lasso(alpha=.35, normalize=False)
lassBig.fit(X_train2, Y_train)
print('\nR² for the model with many features:')
print(lassBig.score(X_train2, Y_train))
origparams = np.append(lassBig.coef_, lassBig.intercept_)
print('\nParameter estimates for the model with many features:')
print(origparams)

print("")
print('R² for the test data with few features:')
print(lass.score(X_test, Y_test))
print('\nR² for the test data with many features:')
print(lassBig.score(X_test2, Y_test))

R² for the model with few features:
0.733213555375

Parameter estimates for the model with few features:
[ 0.02343462  0.00957195  0.01784776  0.          0.          0.05013214
  0.          0.00674197  0.          0.00895562]

R² for the model with many features:
0.679890180491

Parameter estimates for the model with many features:
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.00135676  0.00268882  0.00297218  0.00612355
  0.          0.00027119  0.10390464]

R² for the test data with few features:
0.722511709555

R² for the test data with many features:
0.753461134895


In [405]:
logisticReg = linear_model.LogisticRegression(penalty='l2', dual=False, tol=0.0001, 
    C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, 
    random_state=None, solver='liblinear', max_iter=100, multi_class='ovr', 
    verbose=0, warm_start=False, n_jobs=1)
logisticRegFit = logisticReg.fit(X_train, Y_train)
print('Logistic Regression: R² for the TRAIN model with few features:')
print(logisticRegFit.score(X_train, Y_train))
origparams = np.append(logisticRegFit.coef_, logisticRegFit.intercept_)
print('\nLogistic Regression: Parameter estimates for the model with few features:')
print(origparams)

# Large number of parameters.
logisticRegBig = linear_model.Lasso(alpha=.35, normalize=False)
logisticRegBigFit = logisticRegBig.fit(X_train2, Y_train)
print('\nR² for the TRAIN model with many features:')
print(logisticRegBigFit.score(X_train2, Y_train))
origparams = np.append(logisticRegBigFit.coef_, logisticRegBigFit.intercept_)
print('\nParameter estimates for the model with many features:')
print(origparams)

print("")
print('R² for the test data with few features:')
print(logisticRegFit.score(X_test, Y_test))
print('\nR² for the test data with many features:')
print(logisticRegBigFit.score(X_test2, Y_test))

Logistic Regression: R² for the TRAIN model with few features:
0.958944281525

Logistic Regression: Parameter estimates for the model with few features:
[  2.73029848e-01   7.07713870e-02   3.08744365e-01   2.61712642e-01
   1.29082722e-03   3.25903482e-01  -6.89581200e-02   1.09527075e-01
   1.48736229e-01  -4.94586128e+00]

R² for the TRAIN model with many features:
0.679890180491

Parameter estimates for the model with many features:
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.00135676  0.00268882  0.00297218  0.00612355
  0.          0.00027119  0.10390464]

R² for the test data with few features:
0.985380116959

R² for the test data with many features:
0.753461134895


/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
